# Julia is Fast
A menudo *benchmarks* es usado para comparar lenguajes. Estos benchmarks pueden dar lugar a largas discusiones, en primer lugar, sobre qué se está evaluando exactamente y, en segundo lugar, qué explica las diferencias. Estas preguntas simples a veces pueden volverse más complicadas de lo que podrías imaginar al principio.

El propósito de este *notebook* es que veas un *benchmark* simple por ti mismo.

Esquema de este *notebook*

- Definir la función de suma
- Implementaciones y benchmarking de sum en...
    - Julia (*built-in*)
    - Julia (*hand-written*)
    - C (*built-in*)
    - python (*hand-written*)
    - python (*numpy*)
    - python (*hand-written*)
- Hacia la explotación del paralelismo con Julia
    - Permitir la asociatividad de punto flotante
    - Haciendo uso de cuatro núcleos a la vez: *built-in*
    - Haciendo uso de cuatro núcleos a la vez: *hand-written*
- Resumen de los *benchmarks*

# `sum`: Una función fácil de entender

Considere la función suma `sum(a)`, la cual calcula
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
Donde $n$ es la longitud `a`.

In [3]:
a = rand(10^7); # vector de 1D de numeros aleatorios, uniformes en el rango [0,1)

In [4]:
sum(a)

5.000953330723257e6

El resultado esperado es ~0.5 * 10^7, ya que la media entre entrada es 0.5

# Evaluación comparativa de algunas formas en algunos idiomas

In [7]:
@time sum(a)

  0.014281 seconds (1 allocation: 16 bytes)


5.000953330723257e6

In [8]:
@time sum(a)

  0.004342 seconds (1 allocation: 16 bytes)


5.000953330723257e6

In [9]:
@time sum(a)

  0.004342 seconds (1 allocation: 16 bytes)


5.000953330723257e6

La macro @time puede generar resultados *ruidosos*, por lo que no es nuestra mejor opción para la evaluación comparativa(*benchmarking*).
Afortunadamente Julia tiene `BenchmarkTools.jl`, un paquete para hacer sencilla y precisa la comparación:

In [10]:
using BenchmarkTools

In [11]:
@benchmark sum($a)

BenchmarkTools.Trial: 710 samples with 1 evaluation.
 Range (min … max):  4.544 ms …  10.668 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.844 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.013 ms ± 890.062 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                        ▁▄█▅▄                                  
  ▂▁▂▂▂▁▁▁▂▃▃▃▂▂▃▃▅▆██▇████████▅▅▃▅▅▃▃▄▂▃▃▂▂▄▃▂▃▃▃▂▃▃▂▃▃▂▂▂▂▃ ▃
  4.54 ms         Histogram: frequency by time          10 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

# 1. Julia *Built-in* (integrado)
Así que ese es el rendimiento de la suma integrada de Julia, pero eso podría estar haciendo muchos trucos para ser rápido, ¡incluso no usar Julia en absoluto en primer lugar! Por supuesto, está escrito en Julia, pero ¿funcionaría si escribimos una implementación sencilla nosotros mismos?

In [12]:
@which sum(a)

sum(a::AbstractArray; dims, kw...) in Base at reducedim.jl:889

Guardemos estos resultados de referencia en un diccionario para que podamos comenzar a realizar un seguimiento de ellos y compararlos en el futuro.

In [13]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 743 samples with 1 evaluation.
 Range (min … max):  4.131 ms …  15.740 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.611 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.695 ms ± 808.673 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                          ▄▆▇▅▇▅█▄▃▅                           
  ▂▂▁▁▂▃▂▃▂▃▃▃▂▂▃▃▄▃▃▄▆▅▆▇████████████▆▄▆▅▄▅▄▅▄▂▄▃▃▂▃▂▃▃▃▃▃▂▃ ▄
  4.13 ms         Histogram: frequency by time        9.06 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [14]:
d = Dict()
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any, Any} with 1 entry:
  "Julia built-in" => 4.1308

## 2. Julia *hand-written*(escrita a mano)

In [15]:
function mysum(A)
    s = 0.0
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [16]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 187 samples with 1 evaluation.
 Range (min … max):  20.253 ms … 30.055 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     27.104 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   26.725 ms ±  1.487 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                         ▁   ▄▁▆█▆█▆           
  ▃▁▁▁▁▁▃▁▁▁▁▁▃▁▁▃▄▁▁▄▁▄▁▁▄▁▄▃▅▄▁▆▃▁▄▁▅▄▆█▄▆▅███████▆▄▄▅▄▆▄▁▃ ▃
  20.3 ms         Histogram: frequency by time        29.1 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [17]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any, Any} with 2 entries:
  "Julia hand-written" => 20.2531
  "Julia built-in"     => 4.1308

Eso es aproximadamente varias veces más lento que la definición integrada. Veremos porqué más adelante.

Pero primero: ¿es esto rápido? ¿Cómo lo sabríamos? Comparémoslo con otros idiomas...

# 3. Lenguaje C

C a menudo se considera el referente dorado: difícil para el ser humano, agradable para la máquina. Llegar a un factor de 2 de C suele ser satisfactorio. No obstante, incluso dentro de C, hay muchos tipos de optimizaciones posibles que un escritor de C puede o no aprovechar.

El autor actual no habla C, por lo que no lee la celda a continuación, pero está feliz de saber que puede poner código C en una sesión de Julia, compilarlo y ejecutarlo. Tenga en cuenta que el """ envuelve una cadena de varias líneas.